<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()
sc = spark.sparkContext

In [3]:
#!rm -rf /content/landing
#!rm -rf /content/bronze
!mkdir -p /content/landing

## Simulate producer:
- extract data from API
- store data as json in the lake
- run task async

In [4]:
import requests
from pyspark.sql.types import *
import json
import datetime
import asyncio

async def ingest_from_api(url: str, table: str, schema: StructType = None):
  response = requests.get(url)
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  if response.status_code == 200:
    data = response.json()
    with open(f"/content/landing/{table}_{int(timestamp)}.json", "w") as f:
      #como estamos a utilizar código python, precisamos de criar a pasta para guardar o conteudo (mkdir acima)
        json.dump(data, f)

async def producer(loop: int, interval_time: int):
  for i in range(loop):
    await ingest_from_api("https://api.carrismetropolitana.pt/vehicles", "vehicles")
    await ingest_from_api("https://api.carrismetropolitana.pt/lines", "lines")
    await asyncio.sleep(interval_time)
    # Garantir que o processo dá a si próprio o periodo de cadência para que também não sejamos bloqueados do lado da API

async def main():
  asyncio.create_task(producer(10, 30))

await main()

- Read from /content/landing as streaming
- store data in memory (for testing)
- store data in the bronze layer

In [6]:
from pyspark.sql.types import *

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

stream = spark.readStream.format("json").schema(vehicle_schema).load("/content/landing/vehicles*")

dedup = stream.dropDuplicates()

In [8]:
# Questionar se estou a trabalhar com Streaming para o Spark
dedup.isStreaming

True

In [9]:
# using memory for testing
try:
  if query.isActive:
    query.stop()
except:
  pass

# Spark não permite que duas querys streaming com o mesmo nome estejam ativas na mesma sessão
# Vai retornar uma query streaming. Se estiver ativa, dámos um stop.

# Estamos a basear-nos no formato memory - mais utilizado para testes.
# Quando uso o memory, tenho vários formatos e neste caso estamos a renomear o nome queryName para vehicles.
query = (dedup.writeStream.format("memory").option("queryName", "vehicles").start())

In [15]:
# Mensagem da query que estava a ser executada
query.status

{'message': 'Getting offsets from FileStreamSource[file:/content/landing/vehicles*]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [21]:
# validar quantos batchs já temos, o que está dentro de cada batch, se já consumimos todos os dados que estao dentro
# dos eventos (a partir desta info: numInputRows)
query.recentProgress

[{'id': '9b6735d5-3fec-421b-8678-477c99ca09d7',
  'runId': '92114148-97d0-4abc-83c7-8d3619a25d83',
  'name': 'vehicles',
  'timestamp': '2024-11-30T09:58:32.632Z',
  'batchId': 0,
  'numInputRows': 4021,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 116.39889998552611,
  'durationMs': {'addBatch': 32634,
   'commitOffsets': 37,
   'getBatch': 255,
   'latestOffset': 173,
   'queryPlanning': 1139,
   'triggerExecution': 34540,
   'walCommit': 226},
  'stateOperators': [{'operatorName': 'dedupe',
    'numRowsTotal': 3607,
    'numRowsUpdated': 3607,
    'allUpdatesTimeMs': 10613,
    'numRowsRemoved': 0,
    'allRemovalsTimeMs': 7,
    'commitTimeMs': 10145,
    'memoryUsedBytes': 1580112,
    'numRowsDroppedByWatermark': 0,
    'numShufflePartitions': 200,
    'numStateStoreInstances': 200,
    'customMetrics': {'loadedMapCacheHitCount': 0,
     'loadedMapCacheMissCount': 0,
     'numDroppedDuplicateRows': 414,
     'stateOnCurrentVersionSizeBytes': 1551312}}],
  'sources': [

In [22]:
spark.sql("select * from vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    183|                1155| IN_TRANSIT_TO| 42|1155|38.917503|   2626|-9.322189|  2626_0_1|  2626_0|            SCHEDULED|       44537|11.666667| 080448|2024-11-30 09:53:02|2626_0_1|160|1|09...|
|      0|                1253|    STOPPED_AT| 42|1253|38.831844|   2017|-9.178019|  2017_0_1|  2017_0|            SCHEDULED|       43844|0.2777778| 070025|2024-11-30 09:52:09|2017_0_1|160|1|09...|
|      0|      

In [ ]:
!rm -rf /content/bronze

In [ ]:
from pyspark.sql.functions import *

# watermark is necessary because of the aggregation
# importante usar o watermark neste ponto pois não queremos perder dados (pode ser, por exemplo, um requisito de negócio)
# Aqui, para todas as operações estou a querer guardar os dados em intervalos de 60s em 60s
transformed = stream.withWatermark("timestamp", "60 seconds")
agg = (transformed
       .groupBy(window(transformed.timestamp, "5 minutes"), col("current_status"))
       .agg(min(transformed.timestamp).alias("init_timestamp"), count("*").alias("count")))

def insert_vehicles(df, batch_id):
  #df2 = df.groupBy("window").pivot("current_status").sum("count")
  # Aqui estamos a utilizar o modo de batch (estamos a utilizar write() e não writeStreaming())
  df.write.format("parquet").mode("append").save("/content/bronze/vehicles")

# using memory for testing
query2 = (agg
          .writeStream
          .outputMode("append")
          # usamos o foreach pois:
          # 1. write the output of a streaming query to data sources that do not have an existing streaming sink.
          # 2. allows you to apply batch functions to the output data of every micro-batch of the streaming query.
          # Temos que passar o método que queremos nestes processos
          .foreachBatch(insert_vehicles)
          .option("checkpointLocation", "/content/bronze/checkpoint")
          .trigger(processingTime='20 seconds')
          .start())

In [ ]:
spark.read.format("parquet").load("/content/bronze/vehicles/*").show(100, False)

+------------------------------------------+--------------+-------------------+-----+
|window                                    |current_status|init_timestamp     |count|
+------------------------------------------+--------------+-------------------+-----+
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|IN_TRANSIT_TO |2024-11-29 00:29:47|5    |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|IN_TRANSIT_TO |2024-11-29 00:14:31|18   |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|IN_TRANSIT_TO |2024-11-29 00:15:02|916  |
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|IN_TRANSIT_TO |2024-11-29 00:20:00|2    |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|INCOMING_AT   |2024-11-29 00:15:01|308  |
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|INCOMING_AT   |2024-11-29 00:29:13|6    |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|INCOMING_AT   |2024-11-29 00:14:29|4    |
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|INCOMING_AT   |2024-11-29 00:20:00|2    |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|STOPPED_AT

## Report
- show vehicles by status in 5-min window time
- one line per window time

In [ ]:
def pivot_data(df: DataFrame):
  result = df.orderBy("init_timestamp").groupBy("window").pivot("current_status").sum("count")
  result.show(100, False)

df = spark.read.format("parquet").load("/content/bronze/vehicles/*")
pivot_data(df)

+------------------------------------------+-----------+-------------+----------+
|window                                    |INCOMING_AT|IN_TRANSIT_TO|STOPPED_AT|
+------------------------------------------+-----------+-------------+----------+
|{2024-11-29 00:20:00, 2024-11-29 00:25:00}|2          |2            |NULL      |
|{2024-11-29 00:15:00, 2024-11-29 00:20:00}|308        |916          |295       |
|{2024-11-29 00:10:00, 2024-11-29 00:15:00}|4          |18           |16        |
|{2024-11-29 00:25:00, 2024-11-29 00:30:00}|6          |5            |10        |
+------------------------------------------+-----------+-------------+----------+



In [ ]:
query2.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-48-9446b2336b0e>", line 11, in insert_vehicles
    df.write.format("parquet").mode("append").save("/content/bronze/vehicles")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1463, in save
    self._jwrite.save(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pysp